In [1]:
from MultiMEDal_multimodal_medical.src.datasets.data_transform import (
    build_transform_dict,
    build_transform_dict_mamm,
    build_transform_dict_blip2,
    build_transform_dict_openclip
)
from MultiMEDal_multimodal_medical.src.datasets.EMBED import EMBED_dataset_tfds, EMBED_tabular_dataset
from torch.utils.data import Dataset, DataLoader
from MultiMEDal_multimodal_medical.src.datasets.preprocessing.prompt_factory import tab2prompt_breast_lesion

/project/hnguyen/hqvo2/miniconda3/envs/multimedal/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transform_dict = build_transform_dict(input_size=224)

In [3]:

test_dataset = EMBED_dataset_tfds(
    data_root="/home/hqvo2/Projects/MultiMEDal_multimodal_medical/data/EMBED/original",
    dataset_dir="images_unique_mapping_tfds",
    incl_bg=False,
    split="test",
    class_type="2classes",
    incl_tabular=True,
    transform=transform_dict['test'],
)

/home/hqvo2/Projects/MultiMEDal_multimodal_medical/src/datasets/EMBED.py:1194: DtypeWarning: Columns (28,31,33,48,49,53,54,55,56,57,59,81,84,85,93,111) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clinical = pd.read_csv(os.path.join(self.data_root, "tables/EMBED_OpenData_clinical.csv"))
/home/hqvo2/Projects/MultiMEDal_multimodal_medical/src/datasets/EMBED.py:1202: DtypeWarning: Columns (115,116,117,118,119,120,130,132,133,134,135,136,138,143,147,148,149) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metadata = pd.read_csv(os.path.join(self.data_root, "tables/EMBED_OpenData_metadata.csv"))
/project/hnguyen/hqvo2/miniconda3/envs/multimedal/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

# Samples: (8485, 87)
# Lesions ID: (8485, 2)
#Samples 1650


In [4]:
test_dataloader = DataLoader(
    test_dataset,
    shuffle=None,
    batch_size=64,
    num_workers=4,
    sampler=None,
    persistent_workers=True,
)

In [9]:
from MultiMEDal_multimodal_medical.src.datasets.data_transform import (
    build_transform_dict,
    build_transform_dict_mamm,
    build_transform_dict_blip2,
    build_transform_dict_openclip,
    build_transform_dict_pubmedclip
)
transform_dict, txt_processors = build_transform_dict_pubmedclip()

In [16]:
for batch_data in test_dataloader:
    processed_text_samples, text_samples = tab2prompt_breast_lesion('open_clip', 'train',
                        batch_data, txt_processors, _context_length=None, _group_age=None)
    print('\n'.join(text_samples))
    break

A patient, whose marital status is married, whose ethnicity is african american  or black, whose ethnic group is unreported, unknown, unavailable, whose age is 42. There are asymmetry findings in left breast. Left breast density is heterogeneously dense. There are masses, with focal asymmetry shape. 
A patient, whose marital status is married, whose ethnicity is african american  or black, whose ethnic group is unreported, unknown, unavailable, whose age is 42. There are asymmetry findings in left breast. Left breast density is heterogeneously dense. There are masses, with focal asymmetry shape. 
A patient, whose marital status is married, whose ethnicity is african american  or black, whose ethnic group is unreported, unknown, unavailable, whose age is 42. There are asymmetry findings in left breast. Left breast density is heterogeneously dense. There are masses, with focal asymmetry shape. 
A patient, whose marital status is married, whose ethnicity is african american  or black, who

In [10]:
_, _, _, _, tissueden, side, massshape, massmargin, massdens, calcfind, calcdistri, marital, ethnicity, ethnic_group, age, asses, findings = train_dataset.clinical_labels.iloc[0]

In [6]:
desc_pd = train_dataset.desc_pd

In [7]:
clinical_labels = desc_pd[["mass", "asymmetry", "arch_distortion", "calc", 'tissueden', 
                        'side', "massshape", "massmargin", 
                        "massdens", "calcfind", "calcdistri", 
                        "MARITAL_STATUS_DESC", "ETHNICITY_DESC", 'ETHNIC_GROUP_DESC', "age_at_study", "asses"]].copy()

In [9]:
clinical_labels = clinical_labels.loc[:,~clinical_labels.columns.duplicated()]

In [10]:
breast_side_dict = {
    'L': 'Left breast',
    'R': 'Right breast',
    'B': 'Both breasts'
}

tissueden_dict = {
    1: "Fatty",
    2: "Scattered",
    3: "Heterogeneously dense",
    4: "Extremely Dense",
    5: "Normal Male",
}

massshape_dict = {
    "G": "Generic",
    "R": "Round",
    "O": "Oval",
    "X": "Irregular",
    "Q": "Questioned architectural distortion",
    "A": "Architectural distortion",
    "T": "Asymmetric tubular structure/solitary dilated duct",
    "N": "Intramammary lymph nodes",
    "B": "Global asymmetry",
    "F": "Focal asymmetry",
    "V": "Developing asymmetry",
    "Y": "Lymph node",
    "S": "Asymmetry",
}

massmargin_dict = {
    "D": "Circumscribed",
    "U": "Obscured",
    "M": "Microlobulated",
    "I": "Indistinct",
    "S": "Spiculated",
}

massdens_dict = {
    "+": "High density",
    "=": "Isodense",
    "-": "Low density",
    "0": "Fat containing",
}

calcfind_dict = {  # be careful with feature '9': Benign
    "A": "Amorphous",
    "9": "Benign",
    "H": "Coarse heterogenous",
    "C": "Coarse popcornlike",
    "D": "Dystrophic",
    "E": "Rim",
    "F": "Fine-linear",
    "B": "Fine linear-branching",
    "G": "Generic",
    "I": "Fine pleomorphic",
    "L": "Large rodlike",
    "M": "Milk of calcium",
    "J": "Oil cyst",
    "K": "Pleomorphic",
    "P": "Punctate",
    "R": "Round",
    "S": "Skin",
    "O": "Lucent centered",
    "U": "Suture",
    "V": "Vascular",
    "Q": "Coarse",
}

calcdistri_dict = {
    "G": "Grouped",
    "S": "Segmental",
    "R": "Regional",
    "D": "Diffuse/scattered",
    "L": "Linear",
    "C": "Clustered",
}



In [11]:
clinical_labels.replace({'side': breast_side_dict}, inplace=True)
clinical_labels.replace({'tissueden': tissueden_dict}, inplace=True)
clinical_labels.replace({'massshape': massshape_dict}, inplace=True)
clinical_labels.replace({'massmargin': massmargin_dict}, inplace=True)
clinical_labels.replace({'massdens': massdens_dict}, inplace=True)
clinical_labels.replace({'calcfind': calcfind_dict}, inplace=True, regex=True)
clinical_labels.replace({'calcdistri': calcdistri_dict}, inplace=True)

clinical_labels.replace({'mass': {0: float("nan"), 1: 'mass'}}, inplace=True)
clinical_labels.replace({'asymmetry': {0: float("nan"), 1: 'asymmetry'}}, inplace=True)
clinical_labels.replace({'arch_distortion': {0: float("nan"), 1: 'architectural distortion'}}, inplace=True)
clinical_labels.replace({'calc': {0: float("nan"), 1: 'calcification'}}, inplace=True)

In [12]:
clinical_labels['findings'] = clinical_labels[['mass', 'asymmetry', 'arch_distortion', 'calc']].apply(
    lambda x: ', '.join(x.dropna().astype(str)),
    axis=1
)

In [24]:
clinical_labels["massshape"].value_counts()

Focal asymmetry                        251
Questioned architectural distortion     99
Asymmetry                               98
Oval                                    71
Irregular                               49
Architectural distortion                39
Generic                                 26
Round                                   22
Developing asymmetry                     4
Global asymmetry                         1
Name: massshape, dtype: int64

In [29]:
clinical_labels["tissueden"].value_counts()

Heterogeneously dense    722
Scattered                561
Extremely Dense          116
Fatty                     48
Name: tissueden, dtype: int64